### Steps and tips
* Take sentences, apply one hot representation, pass through embedding layer, pass thorugh LSTM and then train the model to predict test data
* pad_sequences - input to LSTM / Embedding layer we have to make sure to keep our size input length fixed so we have to add pad either in the start or end
* one_hot - Convert sentences to one hot representation given a vocabulary size
* vocabulary size = 5000
* Onehot representation
* reset_index - since we dropped NaN values
* Data preprocessing - nltk, re, nltk.corpus - stopwords
* Data preprocessing - nltk.stem.porter - PorterStemmer (stemming is done using these libraries)
* re.sub('[^a-zA-Z]', ' ', messages['title'][i]) - we are substituting all the characters excep a-z and A-Z with a blank space
* then lowering of the whole review and then split to remove the stopwords as they are not important
* then all the text data is converted into one hot representation
* Embedding representation
* Before giving to embedding layer we will have to make the size of all inputs same, hence padding (pad_sequence)
* embedding_vector_features = 40
* Sequential - Embedding - LSTM - Dense (AF = sigmoid) - compile (binary_crossentropy, adam, accuracy)
* embedded docs are converted into an array
* train test split on X_final and y_final
* Model training (sign of overfitting since val_accuracy was less than accuracy of training data), adding a dropout
* performance metrics and accuracy
* adding the dropout layer is just a kind of hyperparameter to increase accuracy

In [1]:
! pip install kaggle

In [2]:
! mkdir ~/.kaggle

In [3]:
! cp kaggle.json ~/.kaggle/

In [4]:
! chmod 600 ~/.kaggle/kaggle.json

In [5]:
! kaggle competitions download fake-news

 97% 45.0M/46.5M [00:01<00:00, 40.8MB/s]
100% 46.5M/46.5M [00:01<00:00, 28.5MB/s]


In [6]:
! unzip fake-news.zip

Archive:  fake-news.zip
  inflating: submit.csv              
  inflating: test.csv                
  inflating: train.csv               


In [7]:
! ls

fake-news.zip  kaggle.json  sample_data  submit.csv  test.csv  train.csv


Main code starts below -

In [8]:
import pandas as pd

In [9]:
df = pd.read_csv('/content/train.csv')
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [10]:
# Droping null / NaN values if any
df = df.dropna()

In [11]:
# Extracting independent features
X = df.drop('label', axis = 1)

In [12]:
# Dependent features
y = df['label']

In [13]:
y.value_counts()

0    10361
1     7924
Name: label, dtype: int64

In [14]:
X.shape, y.shape

((18285, 4), (18285,))

In [15]:
import tensorflow as tf
tf.__version__

'2.12.0'

In [16]:
#importing all important libraries
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

#Bidirectional
from tensorflow.keras.layers import Bidirectional

In [17]:
voc_size  = 5000

Onehot Representation

In [18]:
messages = X.copy()

In [19]:
messages['title'][1]

'FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart'

In [20]:
messages.reset_index(inplace = True)

In [21]:
import nltk
import re
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

Data preprocessing

In [22]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
  review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
  review = review.lower()
  review = review.split()

  review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
  review = ' '.join(review)
  corpus.append(review)

In [23]:
corpus[0:5]

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri']

In [24]:
onehot_repr = [one_hot(words, voc_size) for words in corpus]
onehot_repr[0:5]

[[2995, 2593, 849, 1876, 3125, 1239, 1950, 4118, 3168, 961],
 [3736, 3847, 946, 2083, 983, 4385, 160],
 [2200, 3944, 2933, 249],
 [4119, 4577, 4338, 419, 137, 4071],
 [4672, 983, 2991, 2342, 1665, 272, 983, 435, 304, 4404]]

Embedding representation

In [25]:
sent_length = 20
embedded_docs = pad_sequences(onehot_repr, padding = 'pre', maxlen = sent_length)
embedded_docs[0:5]

array([[   0,    0,    0,    0,    0,    0,    0,    0,    0,    0, 2995,
        2593,  849, 1876, 3125, 1239, 1950, 4118, 3168,  961],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0, 3736, 3847,  946, 2083,  983, 4385,  160],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0, 2200, 3944, 2933,  249],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0, 4119, 4577, 4338,  419,  137, 4071],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0, 4672,
         983, 2991, 2342, 1665,  272,  983,  435,  304, 4404]],
      dtype=int32)

In [26]:
embedded_docs[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0, 2995,
       2593,  849, 1876, 3125, 1239, 1950, 4118, 3168,  961], dtype=int32)

Model creation

In [27]:
embedding_vector_features = 40
model = Sequential()
model.add(Embedding(voc_size, embedding_vector_features, input_length = sent_length))
model.add(LSTM(100))
model.add(Dense(1, activation = 'sigmoid'))
model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 40)            200000    
                                                                 
 lstm (LSTM)                 (None, 100)               56400     
                                                                 
 dense (Dense)               (None, 1)                 101       
                                                                 
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________


In [28]:
len(embedded_docs)

18285

In [29]:
y.shape

(18285,)

In [30]:
import numpy as np
X_final = np.array(embedded_docs)
y_final = np.array(y)

In [31]:
X_final.shape, y_final.shape

((18285, 20), (18285,))

In [32]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

Model training

In [33]:
model.fit(X_train, y_train, validation_data = (X_test, y_test), epochs = 10, batch_size = 64)

Epoch 1/10
192/192 [==============================] - 24s 86ms/step - loss: 0.3460 - accuracy: 0.8500 - val_loss: 0.1959 - val_accuracy: 0.9155
Epoch 2/10
192/192 [==============================] - 3s 18ms/step - loss: 0.1477 - accuracy: 0.9440 - val_loss: 0.2146 - val_accuracy: 0.9167
Epoch 3/10
192/192 [==============================] - 3s 14ms/step - loss: 0.1047 - accuracy: 0.9611 - val_loss: 0.2321 - val_accuracy: 0.9099
Epoch 4/10
192/192 [==============================] - 3s 14ms/step - loss: 0.0757 - accuracy: 0.9731 - val_loss: 0.2471 - val_accuracy: 0.9130
Epoch 5/10
192/192 [==============================] - 2s 10ms/step - loss: 0.0569 - accuracy: 0.9809 - val_loss: 0.2592 - val_accuracy: 0.9007
Epoch 6/10
192/192 [==============================] - 2s 9ms/step - loss: 0.0315 - accuracy: 0.9899 - val_loss: 0.3984 - val_accuracy: 0.9059
Epoch 7/10
192/192 [==============================] - 2s 9ms/step - loss: 0.0213 - accuracy: 0.9935 - val_loss: 0.3855 - val_accuracy: 0.9128


Performance and Accuracy

In [34]:
#y_pred = model.predict_classes(X_test)
y_pred = (model.predict(X_test) > 0.5).astype("int32")

189/189 [==============================] - 1s 2ms/step


In [35]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

array([[3119,  300],
       [ 254, 2362]])

In [36]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)

0.9082021541010771

Since we are able to see a bit of overfitting hence we will add a Dropout layer

In [37]:
from tensorflow.keras.layers import Dropout
embedding_vector_features = 40
model=Sequential()
model.add(Embedding(voc_size, embedding_vector_features, input_length=sent_length))
model.add(Dropout(0.2))
model.add(LSTM(100))
model.add(Dropout(0.2))
model.add(Dense(1, activation = 'sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(X_train, y_train, validation_data = (X_test, y_test), epochs = 10, batch_size = 64)

Epoch 1/10
192/192 [==============================] - 14s 63ms/step - loss: 0.3437 - accuracy: 0.8371 - val_loss: 0.2010 - val_accuracy: 0.9168
Epoch 2/10
192/192 [==============================] - 3s 15ms/step - loss: 0.1507 - accuracy: 0.9400 - val_loss: 0.2220 - val_accuracy: 0.9185
Epoch 3/10
192/192 [==============================] - 2s 13ms/step - loss: 0.1091 - accuracy: 0.9582 - val_loss: 0.2153 - val_accuracy: 0.9130
Epoch 4/10
192/192 [==============================] - 4s 23ms/step - loss: 0.0837 - accuracy: 0.9692 - val_loss: 0.2465 - val_accuracy: 0.9133
Epoch 5/10
192/192 [==============================] - 2s 11ms/step - loss: 0.0585 - accuracy: 0.9793 - val_loss: 0.3219 - val_accuracy: 0.9054
Epoch 6/10
192/192 [==============================] - 2s 9ms/step - loss: 0.0433 - accuracy: 0.9855 - val_loss: 0.3291 - val_accuracy: 0.9148
Epoch 7/10
192/192 [==============================] - 2s 10ms/step - loss: 0.0294 - accuracy: 0.9903 - val_loss: 0.3793 - val_accuracy: 0.9089

In [38]:
#y_pred = model.predict_classes(y_test)
y_pred = (model.predict(X_test) > 0.5).astype("int32")

189/189 [==============================] - 1s 4ms/step


In [39]:
confusion_matrix(y_test, y_pred)

array([[3108,  311],
       [ 231, 2385]])

In [40]:
accuracy_score(y_test, y_pred)

0.9101905550952776

In [41]:
#Bidirectional Model

embedding_vector_features = 40
model1 = Sequential()
model1.add(Embedding(voc_size, embedding_vector_features, input_length = sent_length))
model1.add(Bidirectional(LSTM(100)))
model1.add(Dense(1, activation = 'sigmoid'))
model1.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
print(model1.summary())

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 20, 40)            200000    
                                                                 
 bidirectional (Bidirectiona  (None, 200)              112800    
 l)                                                              
                                                                 
 dense_2 (Dense)             (None, 1)                 201       
                                                                 
Total params: 313,001
Trainable params: 313,001
Non-trainable params: 0
_________________________________________________________________
None


In [42]:
X_final.shape, y_final.shape

((18285, 20), (18285,))

In [43]:
model1.fit(X_train, y_train, validation_data = (X_test, y_test), epochs = 10, batch_size = 64)

Epoch 1/10
192/192 [==============================] - 16s 64ms/step - loss: 0.2975 - accuracy: 0.8639 - val_loss: 0.2028 - val_accuracy: 0.9157
Epoch 2/10
192/192 [==============================] - 4s 23ms/step - loss: 0.1433 - accuracy: 0.9457 - val_loss: 0.2135 - val_accuracy: 0.9176
Epoch 3/10
192/192 [==============================] - 2s 13ms/step - loss: 0.1030 - accuracy: 0.9637 - val_loss: 0.2480 - val_accuracy: 0.9112
Epoch 4/10
192/192 [==============================] - 3s 13ms/step - loss: 0.0731 - accuracy: 0.9733 - val_loss: 0.3082 - val_accuracy: 0.9128
Epoch 5/10
192/192 [==============================] - 2s 12ms/step - loss: 0.0487 - accuracy: 0.9846 - val_loss: 0.3367 - val_accuracy: 0.9099
Epoch 6/10
192/192 [==============================] - 3s 14ms/step - loss: 0.0301 - accuracy: 0.9909 - val_loss: 0.4777 - val_accuracy: 0.9077
Epoch 7/10
192/192 [==============================] - 2s 12ms/step - loss: 0.0190 - accuracy: 0.9938 - val_loss: 0.4639 - val_accuracy: 0.902

In [44]:
y_pred1 = (model1.predict(X_test) > 0.5).astype("int32")

189/189 [==============================] - 1s 3ms/step


In [45]:
confusion_matrix(y_test, y_pred1)

array([[3106,  313],
       [ 278, 2338]])

In [46]:
accuracy_score(y_test, y_pred1)

0.9020712510356255

In [47]:
#Bidirectional and dropout

embedding_vector_features = 40
model1=Sequential()
model1.add(Embedding(voc_size, embedding_vector_features, input_length=sent_length))
model1.add(Dropout(0.2))
model1.add(Bidirectional(LSTM(100)))
model1.add(Dropout(0.2))
model1.add(Dense(1, activation = 'sigmoid'))
model1.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(X_train, y_train, validation_data = (X_test, y_test), epochs = 10, batch_size = 64)

Epoch 1/10
192/192 [==============================] - 2s 8ms/step - loss: 0.0103 - accuracy: 0.9964 - val_loss: 0.5175 - val_accuracy: 0.9123
Epoch 2/10
192/192 [==============================] - 2s 9ms/step - loss: 0.0108 - accuracy: 0.9962 - val_loss: 0.5693 - val_accuracy: 0.9125
Epoch 3/10
192/192 [==============================] - 2s 8ms/step - loss: 0.0061 - accuracy: 0.9982 - val_loss: 0.6308 - val_accuracy: 0.9115
Epoch 4/10
192/192 [==============================] - 2s 10ms/step - loss: 0.0071 - accuracy: 0.9975 - val_loss: 0.5910 - val_accuracy: 0.9075
Epoch 5/10
192/192 [==============================] - 2s 12ms/step - loss: 0.0059 - accuracy: 0.9980 - val_loss: 0.6321 - val_accuracy: 0.9095
Epoch 6/10
192/192 [==============================] - 2s 9ms/step - loss: 0.0082 - accuracy: 0.9976 - val_loss: 0.5900 - val_accuracy: 0.9095
Epoch 7/10
192/192 [==============================] - 2s 9ms/step - loss: 0.0062 - accuracy: 0.9979 - val_loss: 0.5608 - val_accuracy: 0.9090
Epoc

In [48]:
model1.fit(X_train, y_train, validation_data = (X_test, y_test), epochs = 10, batch_size = 64)

Epoch 1/10
192/192 [==============================] - 16s 64ms/step - loss: 0.3045 - accuracy: 0.8582 - val_loss: 0.2094 - val_accuracy: 0.9117
Epoch 2/10
192/192 [==============================] - 4s 19ms/step - loss: 0.1506 - accuracy: 0.9414 - val_loss: 0.1968 - val_accuracy: 0.9188
Epoch 3/10
192/192 [==============================] - 4s 20ms/step - loss: 0.1043 - accuracy: 0.9621 - val_loss: 0.2316 - val_accuracy: 0.9185
Epoch 4/10
192/192 [==============================] - 3s 14ms/step - loss: 0.0736 - accuracy: 0.9739 - val_loss: 0.2584 - val_accuracy: 0.9143
Epoch 5/10
192/192 [==============================] - 2s 13ms/step - loss: 0.0491 - accuracy: 0.9832 - val_loss: 0.3087 - val_accuracy: 0.9133
Epoch 6/10
192/192 [==============================] - 2s 12ms/step - loss: 0.0378 - accuracy: 0.9866 - val_loss: 0.3553 - val_accuracy: 0.9047
Epoch 7/10
192/192 [==============================] - 2s 11ms/step - loss: 0.0285 - accuracy: 0.9905 - val_loss: 0.4417 - val_accuracy: 0.912

In [49]:
y_pred1 = (model1.predict(X_test) > 0.5).astype("int32")

189/189 [==============================] - 1s 3ms/step


In [50]:
confusion_matrix(y_test, y_pred1)

array([[3094,  325],
       [ 227, 2389]])

In [51]:
accuracy_score(y_test, y_pred1)

0.9085335542667772